# Infrastructure and Configuration

## Initialize data

In [ ]:
/* Systemorph set of dimensions + mockdata are dispatched to the unconfigured in-memory DataSource */
#!eval-notebook "../Initialization/InitSystemorphToMemory"
/* DataSource is configured and connected to real database */
//#!eval-notebook "../Database/Configure"

In [ ]:
Workspace.InitializeFrom(DataSource);

## Import Dependencies

In [ ]:
#!import "ReportScopes"
#!import "ReportConfigurationAndUtils"

# Report Scope Proposal

In [ ]:
[InitializeScope(nameof(InitAsync))]
public interface FcfReport : IMutableScopeWithStorage<ReportStorage> {
    // Some infrastructure
    private IWorkspace workspace => GetStorage().Workspace;
    private Systemorph.Vertex.Pivot.Reporting.IReportFactory report => GetStorage().Report;
    
    // Basic mutable properties
    (int Year, int Month) ReportingPeriod { get; set; }
    int Year => ReportingPeriod.Year;
    int Month => ReportingPeriod.Month;
    string ReportingNode { get; set; }
    string Scenario { get; set; }
    CurrencyType CurrencyType { get; set; }
    
    ((int Year, int Month) ReportingPeriod, string ReportingNode, string Scenario, CurrencyType) ShowSettings => (ReportingPeriod, ReportingNode, Scenario, CurrencyType);
    
    // Slice and Dice
    IEnumerable<string> RowSlices { get; set; }
    private string[] defaultRowSlices => new string[] { "Novelty", "VariableType" };
    private string[] rowSlices => RowSlices is null ? defaultRowSlices : defaultRowSlices.Concat(RowSlices).ToArray();
    
    IEnumerable<string> ColumnSlices { get; set; }
    private string[] defaultColumnSlices => new string[] { CurrencyGrouper(CurrencyType) };
    private string[] columnSlices => ColumnSlices is null ? defaultColumnSlices : defaultColumnSlices.Concat(ColumnSlices).ToArray();
    
    // Filter
    IEnumerable<(string filterName, string filterValue)> DataFilter { get; set; }
    private (string filterName, object filterValue)[] dataFilter => (DataFilter is null ? Enumerable.Empty<(string, object)>() : DataFilter.Select(x => (x.filterName, (object)x.filterValue))).ToArray();
    
    // Scope Initialization
    async Task InitAsync() {
        var mostRecentPartition = (await workspace.Query<PartitionByReportingNodeAndPeriod>().Where(x => x.Scenario == null).OrderBy(x => x.Year).ThenBy(x => x.Month).ToArrayAsync()).Last();  
        
        ReportingPeriod = (mostRecentPartition.Year, mostRecentPartition.Month);
        ReportingNode = (await workspace.Query<ReportingNode>().Where(x => x.Parent == null).ToArrayAsync()).First().SystemName; // TODO: change once user permissions are available
        Scenario = null;
        CurrencyType = CurrencyType.Contractual;
        
        ColumnSlices = "EstimateType".RepeatOnce();

        await GetStorage().InitializeReportIndependentCacheAsync();
    }
    
    // Data update in Storage and Report Generation
    private async Task<GridOptions> GetReportTaskAsync() {
        await GetStorage().InitializeAsync(ReportingPeriod, ReportingNode, Scenario, CurrencyType);
    
        var identities = GetStorage().GetIdentities(ReportingPeriod, ReportingNode, Scenario, CurrencyType);
        var dataCube = GetScopes<Fcf>(identities).Aggregate().Fcf.Filter(dataFilter);
        
        return report.ForDataCube(dataCube)
            .WithQuerySource(workspace)
            .SliceRowsBy(rowSlices)
            .SliceColumnsBy(columnSlices)
            .ReportGridOptions()
            .ToReport();
    }

    Task<GridOptions> ToReportAsync => GetReportTaskAsync();
 }

# Playground

In [ ]:
var reportStorage = new ReportStorage(Workspace, Report);

In [ ]:
var fcfReport = Scopes.ForSingleton().WithStorage(reportStorage).ToScope<FcfReport>();

In [ ]:
fcfReport.ReportingNode = "CH";

In [ ]:
fcfReport.ShowSettings

In [ ]:
(await fcfReport.ToReportAsync) with {Height = 250}

In [ ]:
fcfReport.ReportingPeriod = (2020, 12);

In [ ]:
fcfReport.ShowSettings

In [ ]:
(await fcfReport.ToReportAsync) with {Height = 250}

In [ ]:
fcfReport.ReportingNode = "G";

In [ ]:
fcfReport.ShowSettings

In [ ]:
(await fcfReport.ToReportAsync) with {Height = 250}

In [ ]:
fcfReport.ReportingPeriod = (2021, 3);

In [ ]:
fcfReport.ShowSettings

In [ ]:
(await fcfReport.ToReportAsync) with {Height = 250}

In [ ]:
fcfReport.ReportingNode = "CH";

In [ ]:
fcfReport.ColumnSlices = "EconomicBasis".RepeatOnce();

In [ ]:
fcfReport.DataFilter = new [] {("VariableType", "!BOP"), ("VariableType", "!EOP")};

In [ ]:
(await fcfReport.ToReportAsync) with {Height = 500}